In [ ]:
# Autoload modules
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

sys.path.append("../")

from qiskit.circuit.library import RealAmplitudes, EfficientSU2
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.minimum_eigensolvers import SamplingVQE
from qiskit_ibm_runtime import Sampler
from qiskit.primitives import Sampler as LocalSampler
import matplotlib.pyplot as plt
import numpy as np
from qufold import (
    MiyazawaJerniganInteraction,
    Peptide,
    ProteinFoldingProblem,
    PenaltyParameters,
)
from qiskit.quantum_info import SparsePauliOp


In [ ]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
 
service = QiskitRuntimeService()
backend = service.least_busy(simulator=False, operational=True)
# backend = service.backend("ibm_cleveland")


pm= generate_preset_pass_manager(optimization_level=3, backend=backend)

In [ ]:
def build_pf(main_seq: str):
    """Builds the protein folding problem for the given sequence."""
    # Define the interaction
    side_chains = [""] * len(main_seq)

    mj_interaction = MiyazawaJerniganInteraction()

    penalty_back = 10
    penalty_chiral = 10
    penalty_1 = 10

    penalty_terms = PenaltyParameters(penalty_chiral, penalty_back, penalty_1)

    peptide = Peptide(main_seq, side_chains)

    protein_folding_problem = ProteinFoldingProblem(peptide, mj_interaction, penalty_terms)

    return protein_folding_problem

In [ ]:
main_chain = "GSNQNNF" # protein primary amino acid sequence
pf = build_pf(main_chain) #creates the PF problem instance

In [ ]:
qubit_op = pf.qubit_op() #creates the problem Hamiltonian

In [ ]:
from qiskit_ibm_runtime import Session, Options, QiskitRuntimeService


options = Options(
    execution={"shots": 5000},
    resilience_level=0,
    transpilation={"skip_transpilation": False},
    optimization_level=3,
    # environment={"job_tags": [f"batch_{i}"]},
)

def run_vqe(qubit_op):
    # set classical optimizer
    optimizer = COBYLA(maxiter=50)

    # set variational ansatz
    ansatz = EfficientSU2(num_qubits=qubit_op.num_qubits,reps=1)
    ansatz_t = pm.run(ansatz)
    print(ansatz_t.num_qubits)
    qubit_op_t = qubit_op.apply_layout(ansatz_t.layout)
    print(qubit_op_t)
    counts = []
    values = []

    def store_intermediate_result(eval_count, parameters, mean, std):
        counts.append(eval_count)
        values.append(mean)
    with Session(backend=backend):
        vqe = SamplingVQE(
            Sampler(options=options),
            ansatz=ansatz_t,
            optimizer=optimizer,
            aggregation=0.1,
            callback=store_intermediate_result)
    
        raw_result = vqe.compute_minimum_eigenvalue(qubit_op_t)

    return raw_result, counts, values

In [ ]:
raw_result, counts, values = run_vqe(qubit_op)

In [ ]:
fig = plt.figure()

plt.plot(counts, values)
plt.ylabel("Conformational Energy")
plt.xlabel("VQE Iterations")

In [ ]:
result = pf.interpret_new(raw_result=raw_result)
print(
    "The bitstring representing the shape of the protein during optimization is: ",
    result.turn_sequence,
)
print("The expanded expression is:", result.get_result_binary_vector())

##

print(f"The folded protein's main sequence of turns is: {result.protein_shape_decoder.main_turns}")
print(f"and the side turn sequences are: {result.protein_shape_decoder.side_turns}")

fig = result.get_figure(title="3dcrd", ticks=False, grid=True)
fig.get_axes()[0].view_init(10, 70)

In [ ]:
result.save_xyz_file(replace=True)
put(main_chain + ".xyz")